# Azure Blob Service Functions

MSTICpy versions > 0.8.9

### Description

This Notebook provides an example of using the Azure Blob Storage features of MSTICpy in order interact with an Azure Blob Storage account.

You must have msticpy installed to run this notebook:
```
%pip install --upgrade msticpy[azure]
```

### Installation and imports

In [6]:
from msticpy.data.azure_blob_storage import AzureBlobStorage
from msticpy.data import data_obfus as mask

The first step is to initalize the AzureBlobStorage client. Each client is specific to a seperate Azure Blob Storage account and you must pass it the account name when initalizing the client. 

In [7]:
abs = AzureBlobStorage(abs_name="abscontainer")
abs.connect()

The first step when working with Azure Blob Storage is identifying the container to store blobs in, we can enumerate all the containers in the account with `.containers()`. Note that only containers you have permissons to access will be shown, the Azure Blob Storage features of MSTICpy use the same authentication methods as other Azure service elements and you can specific `auth_methods` when calling `.connect` to select the required authentication method.

In [8]:
containers = abs.containers()
display(containers.mp_obf.obfuscate(column_map={"etag": "str", "name":"str"}))

Attempting to sign-in with environment variable credentials...
obfuscating columns:
name, etag, 
done


,name,last_modified,etag,public_access,has_immutability_policy,deleted,version,has_legal_hold,metadata
0,bikjdaerb,2020-11-06 21:53:33+00:00,rbjkmanljijmqpqmpjc,None,False,None,None,False,None
1,epfdkmlmqkmqb,2020-11-19 15:22:38+00:00,ooidklonjqooimbjkpb,None,False,None,None,False,None


We can also create new containers with `.create_container()`

In [10]:
abs.create_container(container_name="documenationcontainer")

,name,last_modified,etag,public_access,has_immutability_policy,deleted,version,has_legal_hold
0,documenationcontainer,2020-11-24 17:40:53+00:00,"""0x8D890A0179F6EAA""",None,False,None,None,False


Once we have a container we can write blobs to the container. Blob objects can be of any type but here we are passing a simple string.

In [22]:
abs.upload_to_blob(blob="Here is some test data", container_name="documenationcontainer", blob_name="test-blob")

Upload complete


True

We can list blobs within a container in the same way we listed containers, here you can see the blob we just created.

In [23]:
abs.blobs(container_name="documenationcontainer")

,name,container,snapshot,version_id,is_current_version,blob_type,encrypted_metadata,last_modified,etag,size,...,remaining_retention_days,creation_time,archive_status,encryption_key_sha256,encryption_scope,request_server_encrypted,object_replication_source_properties,object_replication_destination_policy,tag_count,tags
0,test-blob,documenationcontainer,None,None,None,BlobType.BlockBlob,None,2020-11-24 17:47:41+00:00,0x8D890A10A6DC6A3,22,...,None,2020-11-24 17:47:41+00:00,None,None,None,None,[],None,None,None


We can also return the contents of a blob like so:

In [24]:
abs.get_blob(container_name="documenationcontainer", blob_name="test-blob")

b'Here is some test data'

When sharing blobs its often necessary to access a SAS token that allow a user to access a specific blob for a specific amount of time, without having to have explicity permissions on the Azure Blob Storage account.
In the example below we obsfucate the retruned SAS token but it will take the form of a full URL to the blob, with the required SAS token appeneded. By default tokens generated allow access for 7 days but this can be adjusted.

In [26]:
sast = abs.get_sas_token(container_name="documenationcontainer", blob_name="test-blob")
mask.hash_string(sast)

'irrjnoieonelkirmqkerfibemcjbclillbjcncpiooqjklelmqebrambdkjaaqpqirrjnoieonelkirmqkerfibemcjbclillbjcncpiooqjklelmqebrambdkjaaqpqirrjnoieonelkirmqkerfibemcjbclillbjcncpiooqjklelmqebrambdkjaaqpqirrjnoieonelkirmqkerfibemcjbclillbjcncpiooqjklelmqebrambdkjaaqpqirrjnoieonelkirmqkerfibemcjbclillbjcncpiooqjklelmqebrambdkjaaqpq'

Finally we can delete a blob, below we delete the blob we just created and then list all blobs in the container, which returns nothing as it is now empty:

In [20]:
abs.delete_blob(container_name="documenationcontainer", blob_name="test-blob")

True

In [16]:
abs.blobs(container_name="documenationcontainer")